# Funções para criação de gráficos

In [125]:
df = pd.read_csv(r'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP_X\app\data\processed\vagas_norm.csv')
df.head(1)

,id_vaga,data_anuncio,titulo_vaga,titulo_resumo,faixa_salarial,empresa_contratante,estado,cidade,url,descricao,beneficios,regimeContrato,regiao,perfil_vaga,nivel_cargo,salario
0,30077919,2024-08-16,Analista de Dados - Júnior,analista-de-dados-junior,não disponível,CARTÃO DE TODOS,MG,Ipatinga,/vagas/analista-de-dados-junior/30077919/,"Para agregar nosso time, suas principais ativi...","['Seguro Saúde', 'Assistência Médica / Medicin...",CLT (Efetivo),Sudeste,analista,junior,NaN


In [65]:
%%writefile 'C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP_X\app\services\functions_graphs.py'

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import nbformat
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt

def plot_historico_vagas(df):
    '''
    Função para plotar histórico de vagas publicadas por semana
    args:
        df: usar o dataframe vagas_norm
    '''
    df['semana_anuncio'] = pd.to_datetime(df['data_anuncio']).dt.isocalendar()['week']

    df = df.groupby(['semana_anuncio', 'perfil_vaga']).agg({'perfil_vaga': 'count'}).rename(columns={'perfil_vaga': 'count'}).reset_index()

    fig = px.line(df, x='semana_anuncio', y='count', color='perfil_vaga', 
                title='Número de vagas por perfil profissional',
                markers=True)

    fig.show()

def plot_box_salarios(df):
    '''
    args:
        df: usar o dataframe vagas_norm
    '''

    df = df[df['salario'].notna()]

    fig = px.box(df, y="salario", x="perfil_vaga")
    fig.update_traces(quartilemethod="inclusive")
    fig.update_layout(title='Distribuição de salários por perfil profissional')
    fig.show()

def plot_dist_senioridade(df):
    '''
    args:
        df: usar o dataframe vagas_norm
    '''

    df = df.groupby(['perfil_vaga', 'nivel_cargo']).agg(count=('salario', 'size'), media_salario=('salario', 'mean')).reset_index()
    #converter a média salarial para int, exceto o que for nan
    df['media_salario'] = df['media_salario'].apply(lambda x: int(x) if not np.isnan(x) else x)

    #plotando o gráfico
    fig = px.sunburst(df, path=['perfil_vaga', 'nivel_cargo'], values='count',
                    color='media_salario',
                    color_continuous_scale='PuBu',)
    fig.update_layout(title='Distribuição de vagas por nível de cargo')
    fig.update_traces(textinfo='label+percent parent')
    fig.show()

def plot_dist_regiao(df):
    '''
    args:
        df: usar o dataframe vagas_norm
    '''
    df = df.groupby(['perfil_vaga','regiao', 'estado']).size().reset_index(name='count')

    fig = px.sunburst(df, path=['perfil_vaga', 'regiao', 'estado'], values='count')
    fig.update_layout(title='Número de vagas por localidade')
    fig.update_traces(textinfo='label+percent parent')
    fig.show()

def plot_wordcloud(df):
    '''
    Função para plotar um wordcloud com as palavras mais frequentes
    args:
        df: usar o dataframe requisitos
    '''
    df = df.groupby(['tool']).size().reset_index(name='count')

    word_freq = dict(zip(df['tool'], df['count']))

    # Criar o wordcloud
    wc = WordCloud(width=800, height=400, max_words=200, background_color='white').generate_from_frequencies(word_freq)

    # Plotar o wordcloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.show()


def plot_dist_requisitos(df_vagas, df_reqs):
    '''
    Função para plotar um gráfico ternário com os requisitos de ferramentas por perfil profissional
    args:
        df_vagas: usar o dataframe vagas_norm
        df_reqs: usar o dataframe requisitos
    '''

    # Fazer um left join entre os dataframes
    df_vagas = df_vagas[['id_vaga', 'perfil_vaga']]
    df = pd.merge(df_vagas, df_reqs, on='id_vaga', how='left')

    # Contar o uso de cada ferramenta por perfil
    df_counts = df.groupby(['perfil_vaga', 'tool']).size().reset_index(name='count')

    # Limitei apenas às ferramentas que aparecem em pelo menos 3 vagas, pois o gráfico estava muito poluído
    df_counts = df_counts[df_counts['count'] > 2]

    # Pivotar o DataFrame para ter perfis como colunas
    df_pivot = df_counts.pivot(index='tool', columns='perfil_vaga', values='count').fillna(0)

    # Calcular a contagem total de linhas para cada perfil
    total_counts = df.groupby('perfil_vaga').size()

    # Normalizar as contagens pela quantidade total de cada perfil
    for perfil in total_counts.index:
        if perfil in df_pivot.columns:
            df_pivot[perfil] = df_pivot[perfil] / total_counts[perfil]

    # Adicionar um pequeno valor constante para empurrar os pontos para o centro
    epsilon = 0.003  # Valor pequeno
    for perfil in total_counts.index:
        if perfil in df_pivot.columns:
            df_pivot[perfil] += epsilon


    # Adicionar uma coluna com o perfil mais frequente
    df_pivot['winner'] = df_pivot.idxmax(axis=1)

    # Adicionar coluna somando os campos analista, cientista e engenheiro
    df_pivot['total'] = df_pivot['analista'] + df_pivot['cientista'] + df_pivot['engenheiro']

    df_pivot.reset_index(inplace=True)

    # Criar o gráfico ternário
    fig = px.scatter_ternary(df_pivot, 
                            a='cientista', 
                            b='analista', 
                            c='engenheiro', 
                            hover_name='tool',
                            color="winner", 
                            size="total", 
                            size_max=30,
                            title='Requisitos de ferramentas por perfil profissional',
                            text='tool'  # Adiciona rótulos diretamente
                            )

    #ajustar o máximo dos eixos
    fig.update_ternaries(aaxis_min=0, baxis_min=0, caxis_min=0)

    # Atualizar o layout para melhorar a visualização
    fig.update_traces(textposition='top center', 
                    textfont=dict(size=9))  # Ajusta o tamanho da fonte das anotações

    fig.update_layout(height=1000, width=1000)

    # Exibir o gráfico
    fig.show()


Overwriting C:\Users\RodrigoPintoMesquita\Documents\GitHub\PB_TP_X\app\services\functions_graphs.py
